# Solving Dogs vs Cats using Logistic Regression

 - Extract features using pre-trained ResNet
 - Use logistic regression for classification

## Feature extraction

In [1]:
from tensorflow import keras
import tensorflow as tf

from keras.applications import ResNet50
from keras.applications import imagenet_utils
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import glob
import numpy as np
import random
import os

from tqdm import tqdm

/home/yq/miniconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/yq/miniconda3/envs/tf/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
paths = glob.glob("train/*")
random.shuffle(paths)

labels = [os.path.split(x)[-1].split(".")[0] for x in paths]

le = LabelEncoder()
labels = le.fit_transform(labels)
labels = labels.reshape((-1, 1))

In [3]:
# load ResNet, excluding the fully-connected layers
model = ResNet50(weights="imagenet", include_top=False)

features = []
BS = 128
for i in np.arange(0, len(paths), BS):
    pb = paths[i:i+BS]
    lb = labels[i:i+BS]
    xb = []
    
    for xx in pb:
        img = load_img(xx, target_size=(224, 224))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = imagenet_utils.preprocess_input(img)
        xb.append(img)
        
    X = np.vstack(xb)
    feature = model.predict(X, batch_size=BS)
    feature = feature.reshape((feature.shape[0], -1))
    
    features.append(feature)

features = np.vstack(features)    
print(features.shape)

/home/yq/miniconda3/envs/tf/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


(25000, 100352)


## Logistic Regression

In [4]:
class LogisticRegression:
    def __init__(self, reg=0.001):
        self.X = tf.placeholder(tf.float32, (None, 100352), name="X")
        self.y = tf.placeholder(tf.float32, (None, 1), name="y")
        C= tf.constant(reg)
        
        self.logits, self.loss = self.cost_function(self.X, self.y, C)
        self.predictions = tf.round(self.logits)
        correct_predictions = tf.equal(self.predictions, self.y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

        opt = tf.train.GradientDescentOptimizer(0.001)
        self.train_op = opt.minimize(self.loss)
        
    def cost_function(self, X, y, C):
        weights = tf.Variable(tf.zeros(shape=(100352,1)), name="theta")
        bias = tf.Variable(tf.zeros(shape=(1)), name="bias")

        logits = tf.nn.sigmoid(tf.nn.xw_plus_b(X, weights, bias))

        cost = tf.reduce_mean(tf.square(logits - y)) + tf.reduce_mean(C*weights)

        return logits, cost

### Training the logistic regressor

In [5]:
Xtr, Xte, ytr, yte = train_test_split(features, labels, test_size=2500)

In [6]:
model = LogisticRegression()

saver = tf.train.Saver()
init = tf.global_variables_initializer()

In [7]:
with tf.Session() as sess:
    init.run()
    
    for epoch in range(3):
        idxs = np.random.permutation(len(Xtr))
        t = tqdm(np.array_split(idxs, len(Xtr)//BS))
        
        for idx in t:
            xb, yb = Xtr[idx], ytr[idx]
            _, b_loss, b_acc = sess.run([model.train_op, model.loss, model.accuracy], feed_dict={model.X: xb, model.y: yb})
            
            val_loss = 0.0
            val_acc = 0.0
            count = 0

            for i in range(0, len(Xte), BS):
                xb, yb = Xte[i:i+BS], yte[i:i+BS]
                vloss, vacc = sess.run([model.loss, model.accuracy], feed_dict={model.X: xb, model.y: yb})

                val_loss += vloss
                val_acc += vacc

                count += 1
                
            val_loss /= count
            val_acc/= count
            
            t.set_description(str(epoch+1))
            t.set_postfix(loss=b_loss, acc=b_acc, val_loss=val_loss, val_acc=val_acc)
        
            
    saver.save(sess, "save/logistic_r.ckpt")

3: 100%|██████████| 175/175 [01:46<00:00,  1.64it/s, acc=1, loss=0.00149, val_acc=0.987, val_loss=0.00966]    


### Test it on the test set for submission!

In [8]:
import pandas as pd
import itertools

In [9]:
# loading the test images and performing feature extraction
# copy and paste from the above cell, yes this is messy

paths = glob.glob("test1/*")

model = ResNet50(weights="imagenet", include_top=False)

features = []
BS = 128
for i in np.arange(0, len(paths), BS):
    pb = paths[i:i+BS]
    lb = labels[i:i+BS]
    xb = []
    
    for xx in pb:
        img = load_img(xx, target_size=(224, 224))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = imagenet_utils.preprocess_input(img)
        xb.append(img)
        
    X = np.vstack(xb)
    feature = model.predict(X, batch_size=BS)
    feature = feature.reshape((feature.shape[0], -1))
    
    features.append(feature)

features = np.vstack(features)    
print(features.shape)

/home/yq/miniconda3/envs/tf/lib/python3.6/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


(12500, 100352)


In [15]:
tf.reset_default_graph()

model = LogisticRegression()
saver = tf.train.Saver()
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    saver.restore(sess, "save/logistic_r.ckpt")
    
    preds = []
    
    for i in range(0, len(features), BS):
        xb = features[i:i+BS]
        pred = sess.run(model.predictions, feed_dict={model.X: xb})
        pred = np.squeeze(pred, -1)
        preds.append(pred)
        
preds = list(itertools.chain.from_iterable(preds))

INFO:tensorflow:Restoring parameters from save/logistic_r.ckpt


In [16]:
# writing it to csv file for submission

df = pd.DataFrame({"id": np.arange(1, len(features)+1), "label": preds})
df.to_csv("submission_lr.csv", index=False)